In [26]:
from utils import *
from configs import *
from copy import deepcopy
from evaluate import *

from fit_pMAE import *
from fit_ReMasker import *

import random

In [27]:
set_all_seeds(42)

In [28]:
args = get_args_parser().parse_args([])

#### data

In [34]:
# CONFIGS
basedir = './'
dset = 'diabetes'
# key 1: pattern (full / quasi)
missing_pattern = 'full' # quasi
# key 2: seed (0~9)
seed = 2

print( res_new.keys() )
print( res_new['quasi'].keys() )
print( res_new['full'].keys() )

# LOAD
proc_new = torch.load(f'{basedir}/amputation/{dset}/new_proc.pkl')

X_num = proc_new['X_num']
X_cat = proc_new['X_cat']
d_numerical = proc_new['d_numerical']

# full, quasi
res_new = torch.load(f'{basedir}/amputation/{dset}/amputed.pkl')
res_new_ = res_new[missing_pattern][seed]

X_init_new = res_new_['X_init'].float()
mask_new = res_new_['mask'] == 1

X_incomp_new = res_new_['X_incomp'].float()

n, d = X_init_new.shape
d_numerical = proc_new['d_numerical']
miss_cols_new = [i for i in range(d) if i not in res_new_['full_cols']]

dict_keys(['d', 'd_numerical', 'quasi', 'full'])
dict_keys([9999, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
dict_keys([9999, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


In [14]:
if (n < 1000):
    batch_size = 128
elif (n < 2500):
    batch_size = 256
elif (n <5000):
    batch_size = 512
elif (n <10000):
    batch_size = 1024
elif (n <20000):
    batch_size = 2048
else:
    batch_size = 4096

#### model

In [15]:
print(args)

Namespace(dataset='california', batch_size=64, max_epochs=600, accum_iter=1, mask_ratio=0.5, embed_dim=32, depth=6, decoder_depth=4, num_heads=4, mlp_ratio=4.0, encode_func='linear', norm_field_loss=False, weight_decay=0.05, lr=None, blr=0.001, min_lr=1e-05, warmup_epochs=40, path='/data/', exp_name='test', device='cuda', seed=666, overwrite=True, pin_mem=True)


In [16]:
model_pmae = ProportionalMasker(args)
model_pmae.batch_size = batch_size
model_pmae.device = 'cuda:0'
model_pmae.max_epochs = 300


model_remasker = ReMasker(args)
model_remasker.batch_size = batch_size
model_remasker.device = 'cuda:0'
model_remasker.max_epochs = 300

In [17]:
# pMAE

model_pmae.old_loss = False
model_pmae.block_mlp = 0 # Mixer -0 / Transformer - None
model_pmae.new_imp = True

X_imputed1 = model_pmae.fit(pd.DataFrame(X_incomp_new))
                                                                                         

0.5
missing col index: [ 0  1  3  4  5  6  7  8  9 10]
p_obs mean:  tensor([0.4592, 0.4592, 1.0000, 0.5841, 0.9134, 0.8491, 0.9379, 0.5704, 0.1783,
        0.8087, 0.2614], device='cuda:0')
Loss at epoch 0: 0.5248087765593872
1 epoch took: 0.3287644386291504 sec
Loss at epoch 20: 0.22578491996633582
Loss at epoch 40: 0.13299243206027156
Loss at epoch 60: 0.10124142801581933
Loss at epoch 80: 0.09133690039986625
Loss at epoch 100: 0.08296535471975232
Loss at epoch 120: 0.08094799150799636
Loss at epoch 140: 0.07741889546265378
Loss at epoch 160: 0.07657723842486565
Loss at epoch 180: 0.07482132344895472
Loss at epoch 200: 0.07165924507498256
Loss at epoch 220: 0.072050422307533
Loss at epoch 240: 0.07409956212645838
Loss at epoch 260: 0.07273634169450137
Loss at epoch 280: 0.07100606895919157


In [18]:
# ReMasker

model_remasker.new_imp = False
model_remasker.device = 'cuda:0'
X_imputed2 = model_remasker.fit(pd.DataFrame(X_incomp_new))
                                                                                         

0.5
missing col index: [ 0  1  3  4  5  6  7  8  9 10]
p_obs mean:  tensor([0.4592, 0.4592, 1.0000, 0.5841, 0.9134, 0.8491, 0.9379, 0.5704, 0.1783,
        0.8087, 0.2614], device='cuda:0')
Loss at epoch 0: 0.5553882989509623
1 epoch took: 0.24612069129943848 sec
Loss at epoch 20: 0.16816428542999248
Loss at epoch 40: 0.158639907305649
Loss at epoch 60: 0.13560395440617695
Loss at epoch 80: 0.12286396092922502
Loss at epoch 100: 0.11444827518126037
Loss at epoch 120: 0.10914467717705668
Loss at epoch 140: 0.11234698259330217
Loss at epoch 160: 0.11071328291080863
Loss at epoch 180: 0.1070439837241196
Loss at epoch 200: 0.10533944344890986
Loss at epoch 220: 0.10484647035385843
Loss at epoch 240: 0.10446862002525456
Loss at epoch 260: 0.10317122700907687
Loss at epoch 280: 0.10076127233182681


#### evaluation
- d: number of columns (total)
- d_numerical: number of columns (numerical only)
- X_init_new: ground truth
- X_imputed: imputed value
- mask_new: missing mask (1: miss)
- miss_cols: target columns that are missing
- cat_exists: whether there are categorical variables

In [21]:
# pMAE

X_imputed = X_imputed1

score_dict = evaluator(X_init_new, X_imputed, mask_new, d, d_numerical, 
                       miss_cols = miss_cols_new, 
                       cat_exists = (d - d_numerical > 0) )

print(score_dict)

{'imp_acc': 0.2079109251499176, 'R2(numerical)': 0.2079109251499176, 'Acc(categorical)': nan, 'WD': 0.1298959109789264, 'RMSE': 0.19498909817842003, 'RMSE(num)': 0.19498909817842003, 'RMSE(cat)': nan}


In [22]:
# ReMasker

X_imputed = X_imputed2

score_dict = evaluator(X_init_new, X_imputed, mask_new, d, d_numerical, 
                       miss_cols = miss_cols_new, 
                       cat_exists = (d - d_numerical > 0) )
print(score_dict)

{'imp_acc': 0.04818350821733475, 'R2(numerical)': 0.04818350821733475, 'Acc(categorical)': nan, 'WD': 0.12344947032736571, 'RMSE': 0.18287797114933785, 'RMSE(num)': 0.18287797114933785, 'RMSE(cat)': nan}
